In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
houses = pd.read_csv('../datasets/train.csv')
#test = pd.read_csv('../datasets/test.csv')
#sample_sub_reg = pd.read_csv('../datasets/sample_sub_reg.csv')

In [3]:
houses.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


## 80 columns y=houses['SalePrice']

## Datatypes 

In [4]:
houses.columns =houses.columns.str.lower().str.replace(' ','_')

In [5]:
features = list(houses.columns)[:-1]

## Part 1: Missing values: What is missing? should I drop it, keep it as nan, set it to zero? 

Variable Name | Description | Units
--- | --- | ---
saleprice | --- | ---
id | --- | ---
pid | --- | ---
ms_subclass | --- | ---
ms_zoning | --- | ---
lot_frontage | --- | ---
lot_area | --- | ---
street | --- | ---
alley | --- | ---
lot_shape | --- | ---
land_contour | --- | ---
utilities | --- | ---
lot_config | --- | ---
neighborhood | --- | ---
condition_1 | --- | ---
condition_2 | --- | ---
bldg_type | --- | ---
house_style | --- | ---
overall_qual | --- | ---
overall_cond | --- | ---
year_built | --- | ---
year_remod/add | --- | ---
roof_style | --- | ---
roof_matl | --- | ---
exterior_1st | --- | ---
exterior_2nd | --- | ---
mas_vnr_type | --- | ---
mas_vnr_area | --- | ---
exter_qual | --- | ---
exter_cond | --- | ---
foundation | --- | ---
bsmt_qual | --- | ---
bsmt_cond | --- | ---
bsmt_exposure | --- | ---
bsmtfin_type_1 | --- | ---
bsmtfin_sf_1 | --- | ---
bsmtfin_type_2 | --- | ---
bsmtfin_sf_2 | --- | ---
bsmt_unf_sf | --- | ---
total_bsmt_sf | --- | ---
heating | --- | ---
heating_qc | --- | ---
central_air | --- | ---
electrical | --- | ---
1st_flr_sf | --- | ---
2nd_flr_sf | --- | ---
low_qual_fin_sf | --- | ---
gr_liv_area | --- | ---
bsmt_full_bath | --- | ---
bsmt_half_bath | --- | ---
full_bath | --- | ---
half_bath | --- | ---
bedroom_abvgr | --- | ---
kitchen_abvgr | --- | ---
kitchen_qual | --- | ---
totrms_abvgrd | --- | ---
functional | --- | ---
fireplaces | --- | ---
fireplace_qu | --- | ---
garage_type | --- | ---
garage_yr_blt | --- | ---
garage_finish | --- | ---
garage_cars | --- | ---
garage_area | --- | ---
garage_qual | --- | ---
garage_cond | --- | ---
paved_drive | --- | ---
wood_deck_sf | --- | ---
open_porch_sf | --- | ---
enclosed_porch | --- | ---
3ssn_porch | --- | ---
screen_porch | --- | ---
pool_area | --- | ---
pool_qc | --- | ---
fence | --- | ---
misc_feature | --- | ---
misc_val | --- | ---
mo_sold | --- | ---
yr_sold | --- | ---
sale_type | --- | ---



## Time to get serious. before starting this at 48k MSE for just using ['overall_qual','overall_cond']
# Below are the notes I took as I went through the process of filling nans (if needed), mapping dictionaries to create ordinal variables based on the few repeated scales found in the data for things like quality/condition
# I am condensing the below steps into functions, minimizing but retaining my commetary, and begining the process of feature engineering after this

#### pool area is fine. 0 means no pool. this will produce a meaningful quality multiplier which is a strategy I plan to implement throughout this project

### garages are tricky
#### if I don't know the year, how can the data be of any value?
##### There is a lot more on garages so maybe I can make sense of it
### logic : all garage attributes depend on this. if 0, no garage. the others are like multipliers

## 0 is wrong for the year. They had stables 2000 years ago
### I am making it equal zero because it and all other **modifier columns** all depend on the presence of the thing they modify or describe. Also,'nan' were only present in this column when garage area = 0
###### I am sure there is a better way to phrase that...

In [6]:

# fence column was split into two ad the original was dropped 
houses = houses.drop(columns=['id','pid'])
houses['fence_private']= houses['fence'].map({'GdPrv':2,'MnPrv':1,'GdWo':0,'MnWw':0,np.nan:0})
houses['fence_wood'] = houses['fence'].map({'GdPrv':0,'MnPrv':0,'GdWo':2,'MnWw':1,np.nan:0})
houses.drop(columns=['fence'], inplace=True)



houses['garage_finish']= houses['garage_finish'].map({'Unf':.5,'RFn':1,'Fin':2,np.nan:0})

houses['garage_qual']= houses['garage_qual'].map({'Po':0,'Fa':.5,'TA':1,'Ex':2,'Gd':1.5,np.nan:0})
houses['garage_cond']= houses['garage_cond'].map({'Po':-.5,'Fa':.5,'TA':1,'Ex':2,'Gd':1.5,np.nan:0})

houses['bsmtfin_type_2']= houses['bsmtfin_type_2'].map({'Unf':.5,'Rec':1.5,'LwQ':1,'BLQ':2,'ALQ':2.5,'GLQ':3,np.nan:0}) # i have decided to make these ordinal!
houses['bsmtfin_type_1']= houses['bsmtfin_type_1'].map({'Unf':.5,'Rec':1.5,'LwQ':1,'BLQ':2,'ALQ':2.5,'GLQ':3,np.nan:0}) # i have decided to make these ordinal!

houses['fireplace_qu']=houses['fireplace_qu'].map({'Ex':2,'Gd':1.5,'TA':1,'Fa':.5,'Po':.25,np.nan:0})
houses['bsmt_qual']=houses['bsmt_qual'].map({'Ex':2,'Gd':1.5,'TA':1,'Fa':.5,'Po':.25,np.nan:0})
houses['bsmt_cond']=houses['bsmt_cond'].map({'Ex':2,'Gd':1.5,'TA':1,'Fa':.5,'Po':.25,np.nan:0})
houses['pool_qc']=houses['pool_qc'].map({'Gd':1.5,'Fa':.5,'TA':1,'Ex':2,np.nan:0}) 

houses['lot_shape']=houses['lot_shape'].map({'Reg':2,'IR1':1,'IR2':1,'IR3':.5,np.nan:0})
houses['land_contour']=houses['land_contour'].map({'lvl':2,'HLS':1,'Bnk':1,'Low':.5,np.nan:0})
houses['exter_qual']= houses['exter_qual'].map({'Po':0,'Fa':.5,'TA':1,'Ex':2,'Gd':1.5,np.nan:0})
houses['exter_cond']= houses['exter_cond'].map({'Po':0,'Fa':.5,'TA':1,'Ex':2,'Gd':1.5,np.nan:0})
houses['heating_qc']= houses['heating_qc'].map({'Po':0,'Fa':.5,'TA':1,'Ex':2,'Gd':1.5,np.nan:0})
houses['kitchen_qual']= houses['kitchen_qual'].map({'Po':0,'Fa':.5,'TA':1,'Ex':2,'Gd':1.5,np.nan:0})
houses['functional']=houses['functional'].map({'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':2,'Sal':1}) # this could be catagorical but i will try to ordinalize it here

houses['mas_vnr_area'] = np.where(houses['mas_vnr_area'].isna(),0,houses['mas_vnr_area'].astype(float)) 
houses['bsmt_full_bath']=np.where(houses['bsmt_full_bath'].isna(),0,houses['bsmt_full_bath'].astype(float)) 
houses['bsmt_half_bath']=np.where(houses['bsmt_half_bath'].isna(),0,houses['bsmt_half_bath'].astype(float)) 
houses['bsmtfin_sf_1']=np.where(houses['bsmtfin_sf_1'].isna(),0,houses['bsmtfin_sf_1'].astype(float)) 
houses['bsmtfin_sf_2']=np.where(houses['bsmtfin_sf_2'].isna(),0,houses['bsmtfin_sf_2'].astype(float)) 
houses['bsmt_unf_sf']=np.where(houses['bsmt_unf_sf'].isna(),0,houses['bsmt_unf_sf'].astype(float)) 
houses['total_bsmt_sf']=np.where(houses['total_bsmt_sf'].isna(),0,houses['total_bsmt_sf'].astype(float)) 
houses['lot_frontage'] = np.where(houses['lot_frontage'].isna(),0,houses['lot_frontage'].astype(float)) 
houses['garage_area'] = np.where(houses['garage_area'].isna(),0,houses['garage_area'].astype(float)) 
houses['garage_yr_blt'] = np.where(houses['garage_yr_blt'].isna(),0,houses['garage_yr_blt'].astype(float))  
houses['garage_cars']=np.where(houses['garage_cars'].isna(),0,houses['garage_cars'].astype(float)) 



## all nan in the above are taken care of. Now time for the non missing values columns

## numeric and ordinal columns are all set. The catagorical columns are to be one-hot-encoded below

In [7]:
#missing_vals # defined before I started hunting nans. Here for visual aid for me

### dummy column party 
#### it makes sense for me to group these

In [9]:
houses_dummy_1= pd.get_dummies(data=houses,columns=['mas_vnr_type','bsmt_exposure','garage_type','alley','misc_feature','land_contour'], drop_first=True,dummy_na=True)# this group of 

houses_dummy_2= pd.get_dummies(data=houses_dummy_1,columns=['ms_subclass','ms_zoning','street','neighborhood','condition_2','bldg_type',
                                                            'house_style','roof_style','roof_matl','exterior_1st','exterior_2nd','foundation',
                                                           'central_air','electrical','sale_type','condition_1','utilities','lot_config','land_slope',
                                                           'heating','paved_drive'], drop_first=True)# this group of 

In [10]:
houses_dummy_2.to_csv('../datasets/cleaned_and_ready1.csv')

In [12]:
df = pd.read_csv('../datasets/cleaned_and_ready1.csv',index_col=['Unnamed: 0'])

In [13]:
df.head()

,lot_frontage,lot_area,lot_shape,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,exter_qual,exter_cond,...,lot_config_FR3,lot_config_Inside,land_slope_Mod,land_slope_Sev,heating_GasW,heating_Grav,heating_OthW,heating_Wall,paved_drive_P,paved_drive_Y
0,0.0,13517,1.0,6,8,1976,2005,289.0,1.5,1.0,...,0,0,0,0,0,0,0,0,0,1
1,43.0,11492,1.0,7,5,1996,1997,132.0,1.5,1.0,...,0,0,0,0,0,0,0,0,0,1
2,68.0,7922,2.0,5,7,1953,2007,0.0,1.0,1.5,...,0,1,0,0,0,0,0,0,0,1
3,73.0,9802,2.0,5,5,2006,2007,0.0,1.0,1.0,...,0,1,0,0,0,0,0,0,0,1
4,82.0,14235,1.0,6,8,1900,1993,0.0,1.0,1.0,...,0,1,0,0,0,0,0,0,0,0


In [14]:
X= df.drop(columns=['saleprice'])
y=df['saleprice']

In [15]:
X.shape

(2051, 220)

In [16]:
df[df.isna()].sum().sort_values(ascending=False)

lot_frontage          0.0
roof_matl_WdShngl     0.0
house_style_SFoyer    0.0
house_style_SLvl      0.0
roof_style_Gable      0.0
                     ... 
ms_subclass_40        0.0
ms_subclass_45        0.0
ms_subclass_50        0.0
ms_subclass_60        0.0
paved_drive_Y         0.0
Length: 221, dtype: float64

In [17]:
missing_vals= [(X[list(X.columns)[i]].isna().mean(),list(X.columns)[i])for i in range(len(list(X.columns)))]

In [18]:
missing_vals

[(0.0, 'lot_frontage'),
 (0.0, 'lot_area'),
 (0.0, 'lot_shape'),
 (0.0, 'overall_qual'),
 (0.0, 'overall_cond'),
 (0.0, 'year_built'),
 (0.0, 'year_remod/add'),
 (0.0, 'mas_vnr_area'),
 (0.0, 'exter_qual'),
 (0.0, 'exter_cond'),
 (0.0, 'bsmt_qual'),
 (0.0, 'bsmt_cond'),
 (0.0, 'bsmtfin_type_1'),
 (0.0, 'bsmtfin_sf_1'),
 (0.0, 'bsmtfin_type_2'),
 (0.0, 'bsmtfin_sf_2'),
 (0.0, 'bsmt_unf_sf'),
 (0.0, 'total_bsmt_sf'),
 (0.0, 'heating_qc'),
 (0.0, '1st_flr_sf'),
 (0.0, '2nd_flr_sf'),
 (0.0, 'low_qual_fin_sf'),
 (0.0, 'gr_liv_area'),
 (0.0, 'bsmt_full_bath'),
 (0.0, 'bsmt_half_bath'),
 (0.0, 'full_bath'),
 (0.0, 'half_bath'),
 (0.0, 'bedroom_abvgr'),
 (0.0, 'kitchen_abvgr'),
 (0.0, 'kitchen_qual'),
 (0.0, 'totrms_abvgrd'),
 (0.0, 'functional'),
 (0.0, 'fireplaces'),
 (0.0, 'fireplace_qu'),
 (0.0, 'garage_yr_blt'),
 (0.0, 'garage_finish'),
 (0.0, 'garage_cars'),
 (0.0, 'garage_area'),
 (0.0, 'garage_qual'),
 (0.0, 'garage_cond'),
 (0.0, 'wood_deck_sf'),
 (0.0, 'open_porch_sf'),
 (0.0, 'enclo

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=512)

In [20]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.6860470171749506

In [21]:
lr.score(X_train,y_train)

0.9355788248537884

## these are the columns with missing values - some columns have values of 'none' that may need to be treated similarly to many of these variables

## I want to break this down by grouped features - If you dont have a basement all the features about basement are zero or nan